In [1]:
import sys
sys.path.append("../../")
sys.path.append("..")
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML

In [94]:
from cellmating.tracking._hausdorf_tracker import Tracker
from cellmating.visualize import animate_images
from cellmating.utils import file_traverse
from cellmating.io import imread, imsave
from cellmating.utils import crop_single_object
from cellmating.visualize import label2rgb

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [136]:
file_root = "/media/wd/T7/data/mating_all/20220903_YAV2104xYAV2105_003/tif/"
# file_root = "/Volumes/T7/data/mating_all/20220903_YAV2104xYAV2105_003/tif/"
file_list = file_traverse(file_root, file_regular=".*._rigid.tif$")

In [214]:
filename = file_list[10]
print(filename)
image = imread(filename)
image = np.moveaxis(image, -1, 1)
print(image.shape)

/media/wd/T7/data/mating_all/20220903_YAV2104xYAV2105_003/tif/YAV2104xYAV2105_005_11_rigid.tif
(241, 4, 1020, 1020)


In [204]:
trace = Tracker(image[:, -1], max_miss=10, min_hist=3, low_ratio=0.5, high_ratio=1, is_acc=False)
_ = trace()
traced_image = trace.to_image()

frame= 1 [1083] [1083]
frame= 2 [1083] [1083]
frame= 3 [1083] [1083, 1084]
frame= 4 [1083, 1108] [1083, 1108]
frame= 5 [1005, 1083, 1108] [1083]
frame= 6 [1083, 1108] [1082, 1083, 1094, 1110]
frame= 7 [1002, 1083, 1108] [1082, 1083, 1096]
frame= 8 [1083, 1108] [1082, 1083, 1095]
frame= 9 [1108] [1093, 1107]
divison: [[1 0 1]]
frame= 10 [1003, 1082, 1108] [1083]
frame= 11 [1082, 1108] [1082, 1087]
divison: [[0 0 1]]
frame= 12 [1082, 1098, 1108] [1084, 1101, 1110]
frame= 13 [1082, 1098] [1086, 1103]
frame= 14 [1082, 1098, 1111] [1084, 1102, 1111]
divison: [[0 0 1]]
frame= 15 [1082, 1098, 1111] [1085, 1102, 1112, 1113]
frame= 16 [1082, 1098] []
frame= 17 [1082, 1098] [1104]
frame= 18 [1082] [1086, 1087]
frame= 19 [1082] []
divison: [[0 0 1]]
frame= 20 [1103] [1086, 1087, 1104]
frame= 21 [] [1086, 1087, 1092]
divison: [[1 0 1]
 [2 2 3]]
frame= 22 [1003, 1110] [1088, 1089, 1100]
frame= 23 [1003, 1110] [1088, 1089, 1100, 1114]
divison: [[0 0 1]]
frame= 24 [1110] [1089, 1090, 1095, 1117]
fram

In [205]:
imsave(filename[:-4]+"_trackv1.tif", traced_image, dtype=np.uint16, imagej=True)

In [169]:
# annotation = label2rgb(traced_image)

In [170]:
# ani = animate_images(image[200:, -1,:,:])
# ani2 = animate_images(annotation[200:])

# html1 = HTML(ani.to_jshtml()).data
# html2 = HTML(ani2.to_jshtml()).data

# # Display the combined animations
# HTML(f"""
# <div style="display: flex;">
#     <div style="flex: 1; margin-right: 10px;">
#         {html1}
#     </div>
#     <div style="flex: 1; margin-left: 10px;">
#         {html2}
#     </div>
# </div>
# """)

In [206]:
from cellmating.utils import crop_object_to_dict
import json
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [207]:
labels = np.unique(traced_image[-1])[1:]
labels = labels[labels>2000]

In [208]:
labels

array([2179, 3184, 3185], dtype=uint16)

In [209]:
roi_dict = crop_object_to_dict(traced_image[-1], length=250, labels=labels)
with open(filename[:-4]+'.json', 'w') as json_file:
    json.dump(roi_dict, json_file, indent=4)

In [213]:
# with open('data.json', 'r') as json_file:
#     loaded_data = json.load(json_file)
# print(loaded_data)

In [212]:
traced_image_stack = np.concatenate((image, traced_image[:,None,:,:]), axis=1)
imsave(filename[:-4]+"_track.tif", traced_image_stack, dtype=np.uint16, imagej=True)

In [14]:
labels = np.unique(traced_image)[1:]
labels = labels[labels>2000]

In [15]:
for la in labels:
    
    print(la)
    x_0, x_1, y_0, y_1 = []

2189
3142
3165
3171
3185
3186


In [54]:
x_all = {}
y_all = {}
for la in labels:
    x, y = np.where(traced_image[-1] == la)
    x_all[la] = [x.min(), x.max(), y.min(), y.max()]
    # x_all.append([x.min(), x.max()])
    # y_all.append([y.min(), y.max()])
    

In [56]:
x_all[2189]

[923, 1002, 484, 531]

In [58]:
2*x.min() - (x.max() - x.min())
2*x.max() - x.min()

259

In [ ]:
import yaml

In [ ]:
import yaml
f = open('meta.yaml', 'w+')
yaml.dump(output_data, f, allow_unicode=True)